In [ ]:
"""

PLOT CYRILLE'S ABUMIP RESULTS AND CORRECT THE NISFs SO THAT IT CORRESPONDS TO MINE

"""

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

READ IN DATA

In [ ]:
inputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/Flux_files_Final/'
#inputpath_GL_corr = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/GL_Fluxes_corrmax/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'

In [ ]:
IMBIE_file = xr.open_dataset(inputpath_BedMachine + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

In [ ]:
geoyear = 2050
pd.read_csv(inputpath_GL+'Flux_ant50.gl1-ssp585_ipsl-cm6a-lr_mar_exp004-sp.m5.abumip_'+str(geoyear)+'.txt', header=5, delimiter='\t\t', index_col=0)

In [ ]:
mnb

In [ ]:
geoyear

In [ ]:
ds_all = xr.Dataset()
#for geoyear in range(2025,2201,25):
all_list = []
for geoyear in range(2025,2201,25):
    all_ABUMIP_list = []
    for mnb in [1,3,5]:
        ABUMIP_list = []
        table_ABUMIP = pd.read_csv(inputpath_GL+'Flux_ant50.gl1-ssp585_ipsl-cm6a-lr_mar_exp004-sp.m'+str(mnb)+'.abumip_'+str(geoyear)+'.txt', header=5, delimiter='\t\t', index_col=0)
        table_ABUMIP_clean = table_ABUMIP.drop(['Unnamed: 3','Flux (Gt/yr)'],axis=1).rename(columns={'Unnamed: 2': 'Flux (Gt/yr)'})
        
        for kisf in IMBIE_file.Nisf:
            flux_Gt_ABUMIP = table_ABUMIP_clean['Flux (Gt/yr)'][IMBIE_file.ID_IMBIE.sel(Nisf=kisf).values]
            ABUMIP_list.append(xr.DataArray(data=np.array(flux_Gt_ABUMIP)).assign_coords({'Nisf': kisf}))
    
        flux_ABUMIP = xr.concat(ABUMIP_list, dim='Nisf').rename('flux_Gt_ABUMIP')
        flux_ABUMIP = xr.concat([xr.DataArray(data=np.array(table_ABUMIP_clean['Flux (Gt/yr)'][124] + table_ABUMIP_clean['Flux (Gt/yr)'][125])).rename('flux_Gt_ABUMIP_'+str(geoyear)).assign_coords({'Nisf': 11, 'ID_IMBIE': 124}),flux_ABUMIP.drop_sel(Nisf=11)], dim='Nisf')
        flux_ABUMIP = xr.concat([xr.DataArray(data=np.array(table_ABUMIP_clean['Flux (Gt/yr)'][66] + table_ABUMIP_clean['Flux (Gt/yr)'][67])).rename('flux_Gt_ABUMIP_'+str(geoyear)).assign_coords({'Nisf': 10, 'ID_IMBIE': 66}),flux_ABUMIP.drop_sel(Nisf=10)], dim='Nisf')

        all_ABUMIP_list.append(flux_ABUMIP.assign_coords({'m':mnb}))
        
    ABUMIP_for_m = xr.concat(all_ABUMIP_list, dim='m')
    all_list.append(ABUMIP_for_m.assign_coords({'geoyear':geoyear}))

ds_all['flux_Gt_ABUMIP'] = xr.concat(all_list, dim='geoyear')
ds_all.to_netcdf(outputpath_GL + 'all_GL_fluxes_varying_m_ElmerIcegeometries.nc')

In [ ]:
flux_all_m = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
flux_all_m